# **Program that subtracts vectors element by element.**
*a. A constant-defined vector, that is a vector from the number of elements known at the compilation stage. The number of elements does not
exceeds the maximum thread block size - 1024 elements.*

*b. A vector of arbitrary size (test up to 1 million elements).
The size is entered as a parameter at program runtime.*

  *For this lab, we need to compare with the
CPU implementation in terms of execution speed, as well as to compare element-by-element the obtained
results to confirm the correctness of GPU-accelerated implementation. For
video card, the total time of copying input data into video memory is measured,
execution of the kernel function and back copying of the resulting data, but not
time of memory allocation and release is measured.*

# **Программа, поэлементно вычитающая векторы.**
*a. Константно заданный вектор, то есть вектор из известного на стадии компиляции количества элементов. Количество элементов не
превышает максимальный размер блока потоков – 1024 элемента.*

*b. Вектор произвольного размера (протестировать до 1 млн. элементов).
Размер вводится в качестве параметра во время работы программы.*

  *Для этой лабораторной нужно сравнить с
ЦП-реализацией по скорости выполнения, а также сравнить поэлементно полученные
результаты, чтобы подтвердить корректность GPU-ускоренной реализации. Для
видеокарты замеряется суммарное время копирования входных данных в видеопамять,
выполнения функции-ядра и обратного копирования результирующих данных, но не
замеряется время выделения и освобождения памяти.*



In [ ]:
!pip install numba

In [ ]:
import numpy as np
from numba import cuda, jit, njit
import numba as nb
import math
import time

In [ ]:
N = 1000000
V1 = np.random.randint(0, 10, size = (N), dtype = np.int64)
V2 = np.random.randint(0, 10, size = (N), dtype = np.int64)

In [ ]:
#kernel
@cuda.jit
def vect_subt(V1, V2, res):
  i = cuda.grid(1)
  if i < len(V1):
    res[i] = V1[i] - V2[i]

In [ ]:
#cpu numba
@nb.jit(parallel = True)
def vect_subt_cpu_numba(VC1, VC2):
  l = VC1.shape[0]
  R = np.zeros(l, dtype = np.int64)
  for i in range(0, l):
    R[i] = VC1[i] - VC2[i]
  return R

In [ ]:
#cpu std
def vect_subt_cpu(VC1, VC2):
  l = VC1.shape[0]
  R = np.zeros(l, dtype = np.int64)
  for i in range(0, l):
    R[i] = VC1[i] - VC2[i]
  return R

In [ ]:
V1_CUDA = cuda.to_device(V1)
V2_CUDA = cuda.to_device(V2)
V_RES_CUDA = cuda.device_array_like(V1)

In [ ]:
block_size = 256
grid_size = math.ceil(N / block_size)
t1 = time.time()
vect_subt[grid_size, block_size](V1_CUDA, V2_CUDA, V_RES_CUDA)
V_RES_LOCAL = V_RES_CUDA.copy_to_host()
t2 = time.time()
print("Время выполнения: " + str(t2 - t1) + " с")
t1 = time.time()
V_RES_CPU = vect_subt_cpu_numba(V1, V2)
t2 = time.time()
print("Время выполнения на CPU с jit: " + str(t2 - t1) + " с")
t1 = time.time()
V_RES_CPU = vect_subt_cpu(V1, V2)
t2 = time.time()
print("Время выполнения на CPU без jit: " + str(t2 - t1) + " с")
np.array_equal(V_RES_CPU, V_RES_LOCAL)


Время выполнения: 0.4163784980773926 с
Время выполнения на CPU с jit: 1.1788570880889893 с
Время выполнения на CPU без jit: 0.8441622257232666 с


True